#Medallion Architecture

Bronze → Silver → Gold using Delta Lake



 Medallion Architecture is about data quality progression, not just layers.

We will Explore Below:-

Why Bronze is append-only

Why Silver is the trust layer

Why Gold is business-specific

How Delta Lake enables incremental, reliable pipelines

How this pattern scales from notebooks → production jobs

#Medallion Architecture : Unity Catalog Structure

![](/Volumes/workspace/ecommerce/ecommerce_data/UC Structure.png)


UC schemas = logical layers

UC volumes = physical storage

Delta tables can later be registered in UC

#Define Paths & Imports

In [0]:
from pyspark.sql import functions as F

# Input source (CSV)
SOURCE_PATH = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv"


# Target Delta table (managed by Unity Catalog)
BRONZE_TABLE = "workspace.bronze.events"



#Read Raw CSV (NO inferSchema)

In [0]:
bronze_df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")  # Important!
    .csv(SOURCE_PATH)
)


#Add Ingestion Metadata (Bronze Responsibility)




In [0]:
bronze_df = (
    bronze_df
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("source_file", F.col("_metadata.file_path"))
)


#Validate Raw Ingestion (No Action Yet)

In [0]:
bronze_df.printSchema()


#Create Bronze Schema (If Not Exists)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze;


#Write to Bronze Delta Table (Managed Table)

In [0]:
(
    bronze_df
    .write
    .format("delta")
    .mode("append")
    .saveAsTable(BRONZE_TABLE)
)


#Validate Bronze Table

In [0]:
%sql
SELECT COUNT(*) FROM workspace.bronze.events;


In [0]:
%sql
DESCRIBE DETAIL workspace.bronze.events;


#Inspect Sample Records

In [0]:
%sql
SELECT *
FROM workspace.bronze.events
LIMIT 10;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
SILVER_PATH = "/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events"


#Read from Bronze Table (Delta)

In [0]:
bronze_df = spark.read.table("workspace.bronze.events")


#Explicit Type Casting

In [0]:
typed_df = (
    bronze_df
    .withColumn("event_time", F.to_timestamp("event_time"))
    .withColumn("price", F.col("price").cast("double"))
)


#Data Quality Filters + Deduplication

In [0]:
clean_df = (
    typed_df
    # Null & range validation
    .filter(F.col("event_time").isNotNull())
    .filter(F.col("price").isNotNull())
    .filter(F.col("price") > 0)
    .filter(F.col("price") < 100000)

    # Remove duplicate events
    .dropDuplicates(["user_session", "event_time"])
)


#Business-Ready Columns

In [0]:
silver_df = (
    clean_df
    .withColumn("event_date", F.to_date("event_time"))
    .withColumn(
        "price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "mid")
         .otherwise("premium")
    )
)


#Define Silver Schema Contract (DO NOT CHANGE)

In [0]:
silver_schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_date", DateType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("category_id", StringType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", StringType(), True),
    StructField("user_session", StringType(), True),
    StructField("ingestion_ts", TimestampType(), True),
    StructField("source_file", StringType(), True),
    StructField("price_tier", StringType(), False)
])


#Enforce Column Order (IMPORTANT)

In [0]:
silver_df = silver_df.select([f.name for f in silver_schema.fields])


In [0]:
silver_df.schema

In [0]:
silver_schema

#Schema Validation 

In [0]:
def normalize(schema):
    return [(f.name, f.dataType.simpleString(), f.nullable) for f in schema.fields]

if normalize(silver_df.schema) != normalize(silver_schema):
    raise Exception("❌ Silver schema contract violated")

print("✅ Silver schema validated successfully")


#Write to Silver Managed Table (Unity Catalog)

In [0]:
(
    silver_df
    .write
    .format("delta")
    .mode("overwrite")   # idempotent for daily rebuilds
    .option("overwriteSchema", "true")
    .saveAsTable("workspace.silver.events")
)


#Data Quality Validation Queries

In [0]:
%sql
SELECT COUNT(*) AS silver_row_count
FROM workspace.silver.events;

In [0]:
%sql
SELECT
  MIN(price) AS min_price,
  MAX(price) AS max_price
FROM workspace.silver.events;


In [0]:
%sql
SELECT price_tier, COUNT(*) 
FROM workspace.silver.events
GROUP BY price_tier;


In [0]:
%sql
DESCRIBE HISTORY workspace.silver.events;


In [0]:
%sql
OPTIMIZE workspace.silver.events
ZORDER BY (event_date, event_type);

In [0]:
%sql
VACUUM workspace.silver.events RETAIN 168 HOURS;


In [0]:
%sql

select count(*) from workspace.silver.events

In [0]:
%sql

select * from workspace.silver.events limit 10

#Build GOLD Aggregation Logic (Initial / Full Load)
##Read Silver Table (Source of Truth)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    DateType, StringType, LongType, DoubleType
)

silver_df = spark.read.table("workspace.silver.events")


#Build Gold Aggregations

In [0]:
gold_df = (
    silver_df
    .groupBy("event_date", "product_id")
    .agg(
        F.countDistinct(F.when(F.col("event_type")=="view", F.col("user_id"))).alias("views"),
        F.countDistinct(F.when(F.col("event_type")=="purchase", F.col("user_id"))).alias("purchases"),
        F.sum(F.when(F.col("event_type")=="purchase", F.col("price"))).alias("revenue")
    )
    .withColumn("conversion_rate", F.when(F.col("views")>0, F.col("purchases")/F.col("views")*100).otherwise(0.0))
)

#Ensure numeric columns are non-nullable

In [0]:
gold_df = gold_df.withColumn("views", F.coalesce(F.col("views"), F.lit(0))) \
                 .withColumn("purchases", F.coalesce(F.col("purchases"), F.lit(0))) \
                 .withColumn("revenue", F.coalesce(F.col("revenue"), F.lit(0.0))) \
                 .withColumn("conversion_rate", F.coalesce(F.col("conversion_rate"), F.lit(0.0)))

#Enforce Gold Schema Contract

In [0]:
gold_schema = StructType([
    StructField("event_date", DateType(), True),      # Nullable=True for groupBy key
    StructField("product_id", StringType(), True),    # Nullable=True for groupBy key
    StructField("views", LongType(), False),
    StructField("purchases", LongType(), False),
    StructField("revenue", DoubleType(), False),
    StructField("conversion_rate", DoubleType(), False)
])


#Validate Schema

In [0]:
def normalize(schema):
    return [(f.name, f.dataType.simpleString(), f.nullable) for f in schema.fields]

if normalize(gold_df.schema) != normalize(gold_schema):
    raise Exception("❌ Gold schema contract violated")
else:
    print("✅ Gold schema validated successfully")


#Load the Gold Table

In [0]:
(
    gold_df
    .write
    .format("delta")
    .mode("overwrite")   # idempotent for daily rebuilds
    .option("overwriteSchema", "true")
    .saveAsTable("workspace.gold.products")
)

# Incremental Load Simulation and Processing

In [0]:
#Simulate Incremental Silver Data
from pyspark.sql import functions as F

# Load full Silver table
silver_df = spark.read.table("workspace.silver.events")

# Simulate incremental data by taking a small sample and modifying user_session
incremental_silver_df = (
    silver_df.limit(100)  # take first 100 rows as "new" incremental data
    .withColumn("user_session", F.concat(F.lit("INC_"), F.col("user_session")))  # make new keys
    .withColumn("price", F.col("price") * 1.1)  # slightly modify price to simulate change
    .withColumn("ingestion_ts", F.current_timestamp())  # new ingestion timestamp
)

# Optional: show sample
incremental_silver_df.show(5, truncate=False)


In [0]:
from pyspark.sql import functions as F

# Step 2: Aggregate Incremental Silver Data for Gold
gold_incremental_df = (
    incremental_silver_df.groupBy("event_date", "product_id")
    .agg(
        # Count distinct users who viewed the product
        F.countDistinct(F.when(F.col("event_type") == "view", F.col("user_id"))).alias("views"),
        
        # Count distinct users who purchased the product
        F.countDistinct(F.when(F.col("event_type") == "purchase", F.col("user_id"))).alias("purchases"),
        
        # Total revenue for purchases
        F.sum(F.when(F.col("event_type") == "purchase", F.col("price"))).alias("revenue")
    )
    # Safe conversion rate calculation: 0 if views = 0
    .withColumn(
        "conversion_rate",
        F.when(F.col("views") > 0, F.col("purchases") / F.col("views") * 100).otherwise(0.0)
    )
)

# Optional: inspect the first 5 rows
gold_incremental_df.show(5, truncate=False)


#Gold Table Upsert using Merge

In [0]:
from delta.tables import DeltaTable

# Load existing Gold table
gold_delta = DeltaTable.forName(spark, "workspace.gold.products")

# Merge incremental data
gold_delta.alias("g").merge(
    gold_incremental_df.alias("s"),
    "g.event_date = s.event_date AND g.product_id = s.product_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()


# Optimize Gold Table

In [0]:
spark.sql("OPTIMIZE workspace.gold.products ZORDER BY (event_date, product_id)")

#Verification

In [0]:
spark.sql("""
SELECT event_date, product_id, views, purchases, revenue, conversion_rate
FROM workspace.gold.products
ORDER BY event_date, product_id
""").show(20, truncate=False)


# Describe Table History

In [0]:
%sql
Describe history workspace.gold.products